# EE 605 Digital Image Processing
# Assignment_3
#### -Karan Khajanchi (21110096)
## Image Filters

In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
def plot_image(img):
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    plt.imshow(img)
    plt.show()

In [ ]:
image1 = cv2.imread("Dataset\image1.png")
image2 = cv2.imread("Dataset\image2.png")
image3 = cv2.imread("Dataset\image3.png")
image4 = cv2.imread("Dataset\image4.png")

Using some OpenCV functions, we define a function to add defocus blur and noise to an image. The function takes the following inputs:
- image = original image.
- ksigma = standard deviation for the gaussian kernel.
- nsigma = standard deviation for the gaussian noise.
- ksize = size of the kernel, the size of the 2D kernel will be (ksize)*(ksize).

In [ ]:
def use_openCV(image,sigma, ksize):
    # Create a circular defocus kernel
    defocus_kernel = cv2.getGaussianKernel(ksize, sigma)

    # Normalize the kernel to make sure the intensity values sum to 1
    defocus_kernel = defocus_kernel / defocus_kernel.sum()

    # Apply the circular defocus kernel using builtin filter2D function which convolves the kernel with the image
    # Since the Gaussian filter is separable, the 2d gaussian kernel can be written as matrix multiplication of A and A'(transpose), where A is the 1D Gaussian
    blurred_image = cv2.filter2D(image, -1, defocus_kernel * defocus_kernel.T)

    # Add Gaussian noise to the blurred image
    noise = np.random.normal(0, sigma, image.shape).astype(np.uint8) #mean is set to zero
    final_img = np.clip(blurred_image + noise, 0, 255)

    return final_img

Now, we will define functions to perform above operations without OpenCV functions. <br>
First, we define a function to add Gaussian noise to any image.

In [ ]:
def add_gaussian_noise(image, std=25):
    # Numpy function to get random normal matrix
    noise = np.random.normal(0, std, image.shape).astype(np.uint8)
    # Add noise to the image
    noisy_image = np.clip(image + noise, 0, 255)

    return noisy_image

Next, we need a function to compute the Gaussian kernel of a given size. For this we use the circularly symmetric gaussian function given as:<br>
$K(x,y) = C exp (-(x^2 + y^2)/2 \sigma^2)$ <br>
The size of the 2D kernel will size*size (where size is given as input, along with the standard deviation of the Gaussian)

In [ ]:
def get_gaussian_kernel_2d(size, sigma):
    kernel = np.fromfunction(
        # The mathematical function used is given above
        lambda x, y: (1/(2*np.pi*sigma**2)) * np.exp(-((x - (size-1)/2)**2 + (y - (size-1)/2)**2) / (2*sigma**2)),
        (size, size),
        dtype=np.float64
    )
    return kernel / np.sum(kernel) # Normalize the kernel before returning

Next we define a function to perform convolution of the kernel with the image. This is a simple implementation using for-loops. For each pixel in the image we have to take one-to-one product of the elements of the kernel and the pixels neighbourhood region. For doing this we also need to pad the image around the edges to handle edge cases or boundary effects. Also, note that this is done for all the three channels (RGB).

In [ ]:
def convolve(image, kernel, ksize):
    # Get the dimensions of the image
    n = len(image)
    m = len(image[0])
    # Getting the width that we need to pad on all four sides, which is half of the kernel size
    r = ksize//2
    # Initiliaze the final image with zeros
    final_img = np.zeros_like(image)
    # Pad the images to avoid boundary effects
    padded_img = np.pad(image, ((r,r), (r,r), (0, 0)), mode='reflect')

    # For each point in the image, keep the kernel centered at that point and take the onr-to-onr product of the kernel and image and place the sum in the image
    for i in range(n):
        for j in range(m):
            for k in range(3):
                x,y,z = i+r, j+r, k
                sum = 0
                for row in range(ksize):
                    for col in range(ksize):
                        sum += kernel[row,col]*padded_img[x - r + row, y - r + col, z]
                final_img[i,j,k] = sum

    return final_img

Finally, we create a function which combines all the tasks and provides the final degraded image as the output. This takes the same inputs as described earlier for ***use_openCV*** function defined above.

In [ ]:
def use_myFunction(image, sigma, ksize):
    kernel = get_gaussian_kernel_2d(ksize,sigma)             # get the gaussian kernel
    blurred_img = convolve(image,kernel,ksize)                # perform convolution and get blurred image
    final_img = add_gaussian_noise(blurred_img, sigma)       # finally add gaussian noise the blurred image

    return final_img

Now, we define functions to calculate the MSE and MAE metrics for any given pair of images.

In [ ]:
def MSE(image1,image2):
    mse = np.mean(np.square(image1.astype(float) - image2.astype(float)))
    return mse

def MAE(image1,image2):
    mae = np.mean(np.abs(image1.astype(float) - image2.astype(float)))
    return mae

Now, call the above functions to add noise to the images in the dataset, and compare how well do our functions match up against the functions in openCV. For computation purposes the size of the kernel is kept 9 at max(Meaning the maximum size of the kernel in any of these computations is 9*9).<br>
The $\sigma$ for gaussian noise and defocus blur increases from **5 to 25** the kernel size in increase from **3 to 11** across each image in the 5 variations of noise.

In [ ]:
test10 = use_myFunction(image1,5,3)
test11 = use_openCV(image1,5,3)
test12 = use_myFunction(image1,10,5)
test13 = use_openCV(image1,10,5)
test14 = use_myFunction(image1,15,7)
test15 = use_openCV(image1,15,7)
test16 = use_myFunction(image1,20,9)
test17 = use_openCV(image1,20,9)
test18 = use_myFunction(image1,25,11)
test19 = use_openCV(image1,25,11)


test20 = use_myFunction(image2,5,3)
test21 = use_openCV(image2,5,3)
test22 = use_myFunction(image2,10,5)
test23 = use_openCV(image2,10,5)
test24 = use_myFunction(image2,15,7)
test25 = use_openCV(image2,15,7)
test26 = use_myFunction(image2,20,9)
test27 = use_openCV(image2,20,9)
test28 = use_myFunction(image2,25,11)
test29 = use_openCV(image2,25,11)


test30 = use_myFunction(image3,5,3)
test31 = use_openCV(image3,5,3)
test32 = use_myFunction(image3,10,5)
test33 = use_openCV(image3,10,5)
test34 = use_myFunction(image3,15,7)
test35 = use_openCV(image3,15,7)
test36 = use_myFunction(image3,20,9)
test37 = use_openCV(image3,20,9)
test38 = use_myFunction(image3,25,11)
test39 = use_openCV(image3,25,11)


test40 = use_myFunction(image4,5,3)
test41 = use_openCV(image4,5,3)
test42 = use_myFunction(image4,10,5)
test43 = use_openCV(image4,10,5)
test44 = use_myFunction(image4,15,7)
test45 = use_openCV(image4,15,7)
test46 = use_myFunction(image4,20,9)
test47 = use_openCV(image4,20,9)
test48 = use_myFunction(image4,25,11)
test49 = use_openCV(image4,25,11)


In [ ]:
# Testing the outputs.
print("Original Image:")
plot_image(image1)
print("Using the function we defined:")
plot_image(test10)
print("Using the OpenCV functions:")
plot_image(test11)
print("Between the 2 noisy images, MSE: " + str(MSE(test10,test11)) + " and MAE: " + str(MAE(test10,test11)))

In [ ]:
print("Now MSE and MAE values for the noisy images produced by openCV and myFunction:")
print("Noisy variants of image1, MSE: " + str(MSE(test10,test11)) + " and MAE: " + str(MAE(test10,test11)))
print("Noisy variants of image2, MSE: " + str(MSE(test20,test21)) + " and MAE: " + str(MAE(test20,test21)))
print("Noisy variants of image3, MSE: " + str(MSE(test30,test31)) + " and MAE: " + str(MAE(test30,test31)))
print("Noisy variants of image4, MSE: " + str(MSE(test40,test41)) + " and MAE: " + str(MAE(test40,test41)))

We can clearly see that the image has been blurred and noise has been added, and that *myFunction* performs similar to the *OpenCV* functions. We also see that the *Mean Absolute Error(MAE)* is very small between the noisy versions of the image but the *Mean Square Error(MSE)* is higher than what you might expect.

The next cell of code can be collapsed and viewed using the clicking on the *View* on the menu above and selecting Expand/Collapse or Show/Hide the selected cells. The reason I have collapsed the code is that they are just arbitrary lines of code where I create a plot on the output using the matplotplotlib. Scroll in the ouput of the cell to see the entire result.

In [ ]:
# @title
fig, axs = plt.subplots(20, 3, figsize=(16, 80))
# Image1 with various noises
axs[0, 0].imshow(cv2.cvtColor(image1, cv2.COLOR_BGR2RGB))
axs[0, 0].set_title('Original')
axs[0, 1].imshow(cv2.cvtColor(test10, cv2.COLOR_BGR2RGB))
axs[0, 1].set_title('myFunc')
axs[0, 2].imshow(cv2.cvtColor(test11, cv2.COLOR_BGR2RGB))
axs[0, 2].set_title('openCV')

axs[1, 0].imshow(cv2.cvtColor(image1, cv2.COLOR_BGR2RGB))
axs[1, 0].set_title('Original')
axs[1, 1].imshow(cv2.cvtColor(test12, cv2.COLOR_BGR2RGB))
axs[1, 1].set_title('myFunc')
axs[1, 2].imshow(cv2.cvtColor(test13, cv2.COLOR_BGR2RGB))
axs[1, 2].set_title('openCV')

axs[2, 0].imshow(cv2.cvtColor(image1, cv2.COLOR_BGR2RGB))
axs[2, 0].set_title('Original')
axs[2, 1].imshow(cv2.cvtColor(test14, cv2.COLOR_BGR2RGB))
axs[2, 1].set_title('myFunc')
axs[2, 2].imshow(cv2.cvtColor(test15, cv2.COLOR_BGR2RGB))
axs[2, 2].set_title('openCV')

axs[3, 0].imshow(cv2.cvtColor(image1, cv2.COLOR_BGR2RGB))
axs[3, 0].set_title('Original')
axs[3, 1].imshow(cv2.cvtColor(test16, cv2.COLOR_BGR2RGB))
axs[3, 1].set_title('myFunc')
axs[3, 2].imshow(cv2.cvtColor(test17, cv2.COLOR_BGR2RGB))
axs[3, 2].set_title('openCV')

axs[4, 0].imshow(cv2.cvtColor(image1, cv2.COLOR_BGR2RGB))
axs[4, 0].set_title('Original')
axs[4, 1].imshow(cv2.cvtColor(test18, cv2.COLOR_BGR2RGB))
axs[4, 1].set_title('myFunc')
axs[4, 2].imshow(cv2.cvtColor(test19, cv2.COLOR_BGR2RGB))
axs[4, 2].set_title('openCV')

# Image2 with various noises
axs[5, 0].imshow(cv2.cvtColor(image2, cv2.COLOR_BGR2RGB))
axs[5, 0].set_title('Original')
axs[5, 1].imshow(cv2.cvtColor(test20, cv2.COLOR_BGR2RGB))
axs[5, 1].set_title('myFunc')
axs[5, 2].imshow(cv2.cvtColor(test21, cv2.COLOR_BGR2RGB))
axs[5, 2].set_title('openCV')

axs[6, 0].imshow(cv2.cvtColor(image2, cv2.COLOR_BGR2RGB))
axs[6, 0].set_title('Original')
axs[6, 1].imshow(cv2.cvtColor(test22, cv2.COLOR_BGR2RGB))
axs[6, 1].set_title('myFunc')
axs[6, 2].imshow(cv2.cvtColor(test23, cv2.COLOR_BGR2RGB))
axs[6, 2].set_title('openCV')

axs[7, 0].imshow(cv2.cvtColor(image2, cv2.COLOR_BGR2RGB))
axs[7, 0].set_title('Original')
axs[7, 1].imshow(cv2.cvtColor(test24, cv2.COLOR_BGR2RGB))
axs[7, 1].set_title('myFunc')
axs[7, 2].imshow(cv2.cvtColor(test25, cv2.COLOR_BGR2RGB))
axs[7, 2].set_title('openCV')

axs[8, 0].imshow(cv2.cvtColor(image2, cv2.COLOR_BGR2RGB))
axs[8, 0].set_title('Original')
axs[8, 1].imshow(cv2.cvtColor(test26, cv2.COLOR_BGR2RGB))
axs[8, 1].set_title('myFunc')
axs[8, 2].imshow(cv2.cvtColor(test27, cv2.COLOR_BGR2RGB))
axs[8, 2].set_title('openCV')

axs[9, 0].imshow(cv2.cvtColor(image2, cv2.COLOR_BGR2RGB))
axs[9, 0].set_title('Original')
axs[9, 1].imshow(cv2.cvtColor(test28, cv2.COLOR_BGR2RGB))
axs[9, 1].set_title('myFunc')
axs[9, 2].imshow(cv2.cvtColor(test29, cv2.COLOR_BGR2RGB))
axs[9, 2].set_title('openCV')

# Image3 with various noises
axs[10, 0].imshow(cv2.cvtColor(image3, cv2.COLOR_BGR2RGB))
axs[10, 0].set_title('Original')
axs[10, 1].imshow(cv2.cvtColor(test30, cv2.COLOR_BGR2RGB))
axs[10, 1].set_title('myFunc')
axs[10, 2].imshow(cv2.cvtColor(test31, cv2.COLOR_BGR2RGB))
axs[10, 2].set_title('openCV')

axs[11, 0].imshow(cv2.cvtColor(image3, cv2.COLOR_BGR2RGB))
axs[11, 0].set_title('Original')
axs[11, 1].imshow(cv2.cvtColor(test32, cv2.COLOR_BGR2RGB))
axs[11, 1].set_title('myFunc')
axs[11, 2].imshow(cv2.cvtColor(test33, cv2.COLOR_BGR2RGB))
axs[11, 2].set_title('openCV')

axs[12, 0].imshow(cv2.cvtColor(image3, cv2.COLOR_BGR2RGB))
axs[12, 0].set_title('Original')
axs[12, 1].imshow(cv2.cvtColor(test34, cv2.COLOR_BGR2RGB))
axs[12, 1].set_title('myFunc')
axs[12, 2].imshow(cv2.cvtColor(test35, cv2.COLOR_BGR2RGB))
axs[12, 2].set_title('openCV')

axs[13, 0].imshow(cv2.cvtColor(image3, cv2.COLOR_BGR2RGB))
axs[13, 0].set_title('Original')
axs[13, 1].imshow(cv2.cvtColor(test36, cv2.COLOR_BGR2RGB))
axs[13, 1].set_title('myFunc')
axs[13, 2].imshow(cv2.cvtColor(test37, cv2.COLOR_BGR2RGB))
axs[13, 2].set_title('openCV')

axs[14, 0].imshow(cv2.cvtColor(image3, cv2.COLOR_BGR2RGB))
axs[14, 0].set_title('Original')
axs[14, 1].imshow(cv2.cvtColor(test38, cv2.COLOR_BGR2RGB))
axs[14, 1].set_title('myFunc')
axs[14, 2].imshow(cv2.cvtColor(test39, cv2.COLOR_BGR2RGB))
axs[14, 2].set_title('openCV')

# Image4 with various noises
axs[15, 0].imshow(cv2.cvtColor(image4, cv2.COLOR_BGR2RGB))
axs[15, 0].set_title('Original')
axs[15, 1].imshow(cv2.cvtColor(test40, cv2.COLOR_BGR2RGB))
axs[15, 1].set_title('myFunc')
axs[15, 2].imshow(cv2.cvtColor(test41, cv2.COLOR_BGR2RGB))
axs[15, 2].set_title('openCV')

axs[16, 0].imshow(cv2.cvtColor(image4, cv2.COLOR_BGR2RGB))
axs[16, 0].set_title('Original')
axs[16, 1].imshow(cv2.cvtColor(test42, cv2.COLOR_BGR2RGB))
axs[16, 1].set_title('myFunc')
axs[16, 2].imshow(cv2.cvtColor(test43, cv2.COLOR_BGR2RGB))
axs[16, 2].set_title('openCV')

axs[17, 0].imshow(cv2.cvtColor(image4, cv2.COLOR_BGR2RGB))
axs[17, 0].set_title('Original')
axs[17, 1].imshow(cv2.cvtColor(test44, cv2.COLOR_BGR2RGB))
axs[17, 1].set_title('myFunc')
axs[17, 2].imshow(cv2.cvtColor(test45, cv2.COLOR_BGR2RGB))
axs[17, 2].set_title('openCV')

axs[18, 0].imshow(cv2.cvtColor(image4, cv2.COLOR_BGR2RGB))
axs[18, 0].set_title('Original')
axs[18, 1].imshow(cv2.cvtColor(test46, cv2.COLOR_BGR2RGB))
axs[18, 1].set_title('myFunc')
axs[18, 2].imshow(cv2.cvtColor(test47, cv2.COLOR_BGR2RGB))
axs[18, 2].set_title('openCV')

axs[19, 0].imshow(cv2.cvtColor(image4, cv2.COLOR_BGR2RGB))
axs[19, 0].set_title('Original')
axs[19, 1].imshow(cv2.cvtColor(test48, cv2.COLOR_BGR2RGB))
axs[19, 1].set_title('myFunc')
axs[19, 2].imshow(cv2.cvtColor(test49, cv2.COLOR_BGR2RGB))
axs[19, 2].set_title('openCV')

for ax in axs.ravel():
    ax.axis('off')

plt.subplots_adjust(wspace=0.1, hspace=0.1)
plt.show()

Now, we define a function to create the wiener filter. The function takes the noisy image as kernel size, sigma, and 'K', the wiener parameter as the input. The wiener filter works accordint to the following formula
$\hat{f}(x, y) = H^*(x, y) \cdot \frac{1}{1 + \frac{S_n(f)}{S_f(f)}} \cdot G(u, v)$ <br>
Here $\frac{S_n(f)}{S_f(f)}$ is the wiener parameter and we denote it with k and vary it for real values of k. G(f) is the frequency domain representation of the noisy image, and it is computed using the fft ftunction in numpy. Similarly, H*(f) is also computed uisng the fft function on the gaussian kernel and then taking its complex conjugate. After computing the frquency domain representation of the image we take the inverse fourier transform and return the restored image.

In [ ]:
def wiener_filter(noisy_img,ksize,sigma,k):
    copy_img = noisy_img.copy()                # Create a copy of the noisy image
    b, g, r = cv2.split(copy_img)              # Split the image into its color channels (BGR order)

    # Normalize each channel to values from 0 to 1
    r = cv2.normalize(r.astype('float32'), None, 0.0, 1.0, cv2.NORM_MINMAX)
    g = cv2.normalize(g.astype('float32'), None, 0.0, 1.0, cv2.NORM_MINMAX)
    b = cv2.normalize(b.astype('float32'), None, 0.0, 1.0, cv2.NORM_MINMAX)

    # Get the gaussian kernel of the specified size
    kernel = cv2.getGaussianKernel(ksize, sigma)

    # Applying the wiener filter through the formula discussed above, for each of the 3 channels separately
    R = np.fft.fft2(r)
    H = np.fft.fft2(kernel,s=r.shape)
    H = np.conj(H) / (np.abs(H)**2 + k)
    R = R*H
    R = np.abs(np.fft.ifft2(R))

    G = np.fft.fft2(g)
    H = np.fft.fft2(kernel,s=g.shape)
    H = np.conj(H) / (np.abs(H)**2 + k)
    G = G*H
    G = np.abs(np.fft.ifft2(G))

    B = np.fft.fft2(b)
    H = np.fft.fft2(kernel,s=b.shape)
    H = np.conj(H) / (np.abs(H)**2 + k)
    B = B*H
    B = np.abs(np.fft.ifft2(B))

    # Merging the three channels to the required image.
    merged_image = cv2.merge([B, G, R])
    merged_image_final = np.clip(merged_image * 255, 0, 255).astype('uint8')

    return merged_image_final

Defining a function to compute the PSNR metric to compare the restored images with the original ones.

In [ ]:
def PSNR(image1, image2, peak=255):
    # Calculating the Mean Squared Error
    mse = MSE(image1,image2)
    # Calculating the Peak Signal Noise Ratio
    psnr = 10*np.log10(peak**2/mse)

    return psnr

Now, we restore the image using the wiener filter that we created. The values of k for each image was chosen after many trials and finally, the value which gave the highest PSNR value was chosen.

In [ ]:
restored11 = wiener_filter(test11,3,5,0.04)
restored13 = wiener_filter(test13,5,10,0.1)
restored15 = wiener_filter(test15,7,15,0.1)
restored17 = wiener_filter(test17,9,20,0.15)
restored19 = wiener_filter(test19,11,25,0.15)

restored21 = wiener_filter(test21,3,5,0.3)
restored23 = wiener_filter(test23,5,10,0.5)
restored25 = wiener_filter(test25,7,15,0.6)
restored27 = wiener_filter(test27,9,20,0.8)
restored29 = wiener_filter(test29,11,25,0.9)

restored31 = wiener_filter(test31,3,5,0.3)
restored33 = wiener_filter(test33,5,10,0.5)
restored35 = wiener_filter(test35,7,15,0.7)
restored37 = wiener_filter(test37,9,20,0.9)
restored39 = wiener_filter(test39,11,25,0.8)

restored41 = wiener_filter(test41,3,5,0.7)
restored43 = wiener_filter(test43,5,10,0.9)
restored45 = wiener_filter(test45,7,15,1.1)
restored47 = wiener_filter(test47,9,20,1.3)
restored49 = wiener_filter(test49,11,25,1.7)

The next cell of code can be collapsed and viewed using the clicking on the *View* on the menu above and selecting Expand/Collapse or Show/Hide the selected cells. The reason I have collapsed the code is that they are just arbitrary lines of code where I create a plot on the output using the matplotplotlib.<br>
Scroll in the ouput of the cell to see the entire result. The first column is the original image, second column is the noisy version of the image, and the third column is restored image which also shows the PSNR value, which is computed by calling the function we defined above.

In [ ]:
# @title
fig, axs = plt.subplots(20, 3, figsize=(16, 80))
# Image1 with restoration
axs[0, 0].imshow(cv2.cvtColor(image1, cv2.COLOR_BGR2RGB))
axs[0, 0].set_title('Original')
axs[0, 1].imshow(cv2.cvtColor(test11, cv2.COLOR_BGR2RGB))
axs[0, 1].set_title('Noisy Image')
axs[0, 2].imshow(cv2.cvtColor(restored11, cv2.COLOR_BGR2RGB))
axs[0, 2].set_title("Restored, PSNR:"+ str(PSNR(image1,restored11)))

axs[1, 0].imshow(cv2.cvtColor(image1, cv2.COLOR_BGR2RGB))
axs[1, 0].set_title('Original')
axs[1, 1].imshow(cv2.cvtColor(test13, cv2.COLOR_BGR2RGB))
axs[1, 1].set_title('Noisy Imgage')
axs[1, 2].imshow(cv2.cvtColor(restored13, cv2.COLOR_BGR2RGB))
axs[1, 2].set_title("Restored, PSNR:"+ str(PSNR(image1,restored13)))

axs[2, 0].imshow(cv2.cvtColor(image1, cv2.COLOR_BGR2RGB))
axs[2, 0].set_title('Original')
axs[2, 1].imshow(cv2.cvtColor(test15, cv2.COLOR_BGR2RGB))
axs[2, 1].set_title('Noisy Imgage')
axs[2, 2].imshow(cv2.cvtColor(restored15, cv2.COLOR_BGR2RGB))
axs[2, 2].set_title("Restored, PSNR:"+ str(PSNR(image1,restored15)))

axs[3, 0].imshow(cv2.cvtColor(image1, cv2.COLOR_BGR2RGB))
axs[3, 0].set_title('Original')
axs[3, 1].imshow(cv2.cvtColor(test17, cv2.COLOR_BGR2RGB))
axs[3, 1].set_title('Noisy Imgage')
axs[3, 2].imshow(cv2.cvtColor(restored17, cv2.COLOR_BGR2RGB))
axs[3, 2].set_title("Restored, PSNR:"+ str(PSNR(image1,restored17)))

axs[4, 0].imshow(cv2.cvtColor(image1, cv2.COLOR_BGR2RGB))
axs[4, 0].set_title('Original')
axs[4, 1].imshow(cv2.cvtColor(test19, cv2.COLOR_BGR2RGB))
axs[4, 1].set_title('Noisy Imgage')
axs[4, 2].imshow(cv2.cvtColor(restored19, cv2.COLOR_BGR2RGB))
axs[4, 2].set_title("Restored, PSNR:"+ str(PSNR(image1,restored19)))

# Image2 with restoration
axs[5, 0].imshow(cv2.cvtColor(image2, cv2.COLOR_BGR2RGB))
axs[5, 0].set_title('Original')
axs[5, 1].imshow(cv2.cvtColor(test21, cv2.COLOR_BGR2RGB))
axs[5, 1].set_title('Noisy Image')
axs[5, 2].imshow(cv2.cvtColor(restored21, cv2.COLOR_BGR2RGB))
axs[5, 2].set_title("Restored, PSNR:"+ str(PSNR(image2,restored21)))

axs[6, 0].imshow(cv2.cvtColor(image2, cv2.COLOR_BGR2RGB))
axs[6, 0].set_title('Original')
axs[6, 1].imshow(cv2.cvtColor(test23, cv2.COLOR_BGR2RGB))
axs[6, 1].set_title('Noisy Image')
axs[6, 2].imshow(cv2.cvtColor(restored23, cv2.COLOR_BGR2RGB))
axs[6, 2].set_title("Restored, PSNR:"+ str(PSNR(image2,restored23)))

axs[7, 0].imshow(cv2.cvtColor(image2, cv2.COLOR_BGR2RGB))
axs[7, 0].set_title('Original')
axs[7, 1].imshow(cv2.cvtColor(test25, cv2.COLOR_BGR2RGB))
axs[7, 1].set_title('Noisy Image')
axs[7, 2].imshow(cv2.cvtColor(restored25, cv2.COLOR_BGR2RGB))
axs[7, 2].set_title("Restored, PSNR:"+ str(PSNR(image2,restored25)))

axs[8, 0].imshow(cv2.cvtColor(image2, cv2.COLOR_BGR2RGB))
axs[8, 0].set_title('Original')
axs[8, 1].imshow(cv2.cvtColor(test27, cv2.COLOR_BGR2RGB))
axs[8, 1].set_title('Noisy Image')
axs[8, 2].imshow(cv2.cvtColor(restored27, cv2.COLOR_BGR2RGB))
axs[8, 2].set_title("Restored, PSNR:"+ str(PSNR(image2,restored27)))

axs[9, 0].imshow(cv2.cvtColor(image2, cv2.COLOR_BGR2RGB))
axs[9, 0].set_title('Original')
axs[9, 1].imshow(cv2.cvtColor(test29, cv2.COLOR_BGR2RGB))
axs[9, 1].set_title('Noisy Image')
axs[9, 2].imshow(cv2.cvtColor(restored29, cv2.COLOR_BGR2RGB))
axs[9, 2].set_title("Restored, PSNR:"+ str(PSNR(image2,restored29)))

# Image3 with various noises
axs[10, 0].imshow(cv2.cvtColor(image3, cv2.COLOR_BGR2RGB))
axs[10, 0].set_title('Original')
axs[10, 1].imshow(cv2.cvtColor(test31, cv2.COLOR_BGR2RGB))
axs[10, 1].set_title('Noisy Image')
axs[10, 2].imshow(cv2.cvtColor(restored31, cv2.COLOR_BGR2RGB))
axs[10, 2].set_title("Restored, PSNR:"+ str(PSNR(image3,restored31)))

axs[11, 0].imshow(cv2.cvtColor(image3, cv2.COLOR_BGR2RGB))
axs[11, 0].set_title('Original')
axs[11, 1].imshow(cv2.cvtColor(test33, cv2.COLOR_BGR2RGB))
axs[11, 1].set_title('Noisy Image')
axs[11, 2].imshow(cv2.cvtColor(restored33, cv2.COLOR_BGR2RGB))
axs[11, 2].set_title("Restored, PSNR:"+ str(PSNR(image3,restored33)))

axs[12, 0].imshow(cv2.cvtColor(image3, cv2.COLOR_BGR2RGB))
axs[12, 0].set_title('Original')
axs[12, 1].imshow(cv2.cvtColor(test35, cv2.COLOR_BGR2RGB))
axs[12, 1].set_title('Noisy Image')
axs[12, 2].imshow(cv2.cvtColor(restored35, cv2.COLOR_BGR2RGB))
axs[12, 2].set_title("Restored, PSNR:"+ str(PSNR(image3,restored35)))

axs[13, 0].imshow(cv2.cvtColor(image3, cv2.COLOR_BGR2RGB))
axs[13, 0].set_title('Original')
axs[13, 1].imshow(cv2.cvtColor(test37, cv2.COLOR_BGR2RGB))
axs[13, 1].set_title('Noisy Image')
axs[13, 2].imshow(cv2.cvtColor(restored37, cv2.COLOR_BGR2RGB))
axs[13, 2].set_title("Restored, PSNR:"+ str(PSNR(image3,restored37)))

axs[14, 0].imshow(cv2.cvtColor(image3, cv2.COLOR_BGR2RGB))
axs[14, 0].set_title('Original')
axs[14, 1].imshow(cv2.cvtColor(test39, cv2.COLOR_BGR2RGB))
axs[14, 1].set_title('Noisy Image')
axs[14, 2].imshow(cv2.cvtColor(restored39, cv2.COLOR_BGR2RGB))
axs[14, 2].set_title("Restored, PSNR:"+ str(PSNR(image3,restored39)))

# Image4 with various noises
axs[15, 0].imshow(cv2.cvtColor(image4, cv2.COLOR_BGR2RGB))
axs[15, 0].set_title('Original')
axs[15, 1].imshow(cv2.cvtColor(test41, cv2.COLOR_BGR2RGB))
axs[15, 1].set_title('Noisy Image')
axs[15, 2].imshow(cv2.cvtColor(restored41, cv2.COLOR_BGR2RGB))
axs[15, 2].set_title("Restored, PSNR:"+ str(PSNR(image4,restored41)))

axs[16, 0].imshow(cv2.cvtColor(image4, cv2.COLOR_BGR2RGB))
axs[16, 0].set_title('Original')
axs[16, 1].imshow(cv2.cvtColor(test43, cv2.COLOR_BGR2RGB))
axs[16, 1].set_title('Noisy Image')
axs[16, 2].imshow(cv2.cvtColor(restored43, cv2.COLOR_BGR2RGB))
axs[16, 2].set_title("Restored, PSNR:"+ str(PSNR(image4,restored43)))

axs[17, 0].imshow(cv2.cvtColor(image4, cv2.COLOR_BGR2RGB))
axs[17, 0].set_title('Original')
axs[17, 1].imshow(cv2.cvtColor(test45, cv2.COLOR_BGR2RGB))
axs[17, 1].set_title('Noisy Image')
axs[17, 2].imshow(cv2.cvtColor(restored45, cv2.COLOR_BGR2RGB))
axs[17, 2].set_title("Restored, PSNR:"+ str(PSNR(image4,restored45)))

axs[18, 0].imshow(cv2.cvtColor(image4, cv2.COLOR_BGR2RGB))
axs[18, 0].set_title('Original')
axs[18, 1].imshow(cv2.cvtColor(test47, cv2.COLOR_BGR2RGB))
axs[18, 1].set_title('Noisy Image')
axs[18, 2].imshow(cv2.cvtColor(restored47, cv2.COLOR_BGR2RGB))
axs[18, 2].set_title("Restored, PSNR:"+ str(PSNR(image4,restored47)))

axs[19, 0].imshow(cv2.cvtColor(image4, cv2.COLOR_BGR2RGB))
axs[19, 0].set_title('Original')
axs[19, 1].imshow(cv2.cvtColor(test49, cv2.COLOR_BGR2RGB))
axs[19, 1].set_title('Noisy Image')
axs[19, 2].imshow(cv2.cvtColor(restored49, cv2.COLOR_BGR2RGB))
axs[19, 2].set_title("Restored, PSNR:"+ str(PSNR(image4,restored49)))

for ax in axs.ravel():
    ax.axis('off')

plt.subplots_adjust(wspace=0.1, hspace=0.1)
plt.show()


for ax in axs.ravel():
    ax.axis('off')

plt.subplots_adjust(wspace=0.1, hspace=0.1)
plt.show()

Now we will compare our restored versions of images, with different restoration algorithms, namely Non-local Means Filter,
Total Variation Regularization, Bilateral Filter, Median Filter, Gaussian Smoothing Filter We will use the inbuilt functions of OpenCV/ Numpy to do this. .

In [ ]:
def nlmeans(image, h = 10, template_size = 7, search_size = 21):
    return cv2.fastNlMeansDenoising(image, None, h, template_size, search_size)

def bilateral(image, diameter = 9, sigma_color = 75, sigma_space = 75):
    return cv2.bilateralFilter(image, diameter, sigma_color, sigma_space)

def median(image, kernel_size = 7):
    return cv2.medianBlur(image, ksize)

def gaussian(image, kernel_size = (7, 7), sigma_x = 0, sigma_y = 0):
    return cv2.GaussianBlur(image, kernel_size, sigma_x, sigma_y)

Now, we have to call the functions for these filters, and give the noisy images as inputs. For this I am choosing the noisy images for which $\sigma$ is equal to 15 and the kernel size 7.

In [ ]:
nlmeans15 = nlmeans(test15)
bilateral15 = bilateral(test15)
median15 = median(test15)
gaussian15 = nlmeans(test15)

nlmeans25 = nlmeans(test25)
bilateral25 = bilateral(test25)
median25 = median(test25)
gaussian25 = nlmeans(test25)

nlmeans35 = nlmeans(test35)
bilateral35 = bilateral(test35)
median35 = median(test35)
gaussian35 = nlmeans(test35)

nlmeans45 = nlmeans(test45)
bilateral45 = bilateral(test45)
median45 = median(test45)
gaussian45 = nlmeans(test45)

In [ ]:
# @title
fig, axs = plt.subplots(4, 7, figsize=(49, 28))
# Image1 with restoration
axs[0, 0].imshow(cv2.cvtColor(image1, cv2.COLOR_BGR2RGB))
axs[0, 0].set_title('Original')
axs[0, 1].imshow(cv2.cvtColor(test15, cv2.COLOR_BGR2RGB))
axs[0, 1].set_title('Noisy Image')
axs[0, 2].imshow(cv2.cvtColor(restored15, cv2.COLOR_BGR2RGB))
axs[0, 2].set_title("Wiener, PSNR:"+ str(PSNR(image1,restored15)))
axs[0, 3].imshow(cv2.cvtColor(nlmeans15, cv2.COLOR_BGR2RGB))
axs[0, 3].set_title("Non-Local Means, PSNR:"+ str(PSNR(image1,nlmeans15)))
axs[0, 4].imshow(cv2.cvtColor(bilateral15, cv2.COLOR_BGR2RGB))
axs[0, 4].set_title("Bilateral, PSNR:"+ str(PSNR(image1,bilateral15)))
axs[0, 5].imshow(cv2.cvtColor(median15, cv2.COLOR_BGR2RGB))
axs[0, 5].set_title("Median, PSNR:"+ str(PSNR(image1,median15)))
axs[0, 6].imshow(cv2.cvtColor(gaussian15, cv2.COLOR_BGR2RGB))
axs[0, 6].set_title("Gaussian , PSNR:"+ str(PSNR(image1,gaussian15)))

axs[1, 0].imshow(cv2.cvtColor(image2, cv2.COLOR_BGR2RGB))
axs[1, 0].set_title('Original')
axs[1, 1].imshow(cv2.cvtColor(test25, cv2.COLOR_BGR2RGB))
axs[1, 1].set_title('Noisy Image')
axs[1, 2].imshow(cv2.cvtColor(restored25, cv2.COLOR_BGR2RGB))
axs[1, 2].set_title("Wiener, PSNR:"+ str(PSNR(image2,restored25)))
axs[1, 3].imshow(cv2.cvtColor(nlmeans25, cv2.COLOR_BGR2RGB))
axs[1, 3].set_title("Non-Local Means, PSNR:"+ str(PSNR(image2,nlmeans25)))
axs[1, 4].imshow(cv2.cvtColor(bilateral25, cv2.COLOR_BGR2RGB))
axs[1, 4].set_title("Bilateral, PSNR:"+ str(PSNR(image2,bilateral25)))
axs[1, 5].imshow(cv2.cvtColor(median25, cv2.COLOR_BGR2RGB))
axs[1, 5].set_title("Median, PSNR:"+ str(PSNR(image2,median25)))
axs[1, 6].imshow(cv2.cvtColor(gaussian25, cv2.COLOR_BGR2RGB))
axs[1, 6].set_title("Gaussian , PSNR:"+ str(PSNR(image2,gaussian25)))

axs[2, 0].imshow(cv2.cvtColor(image3, cv2.COLOR_BGR2RGB))
axs[2, 0].set_title('Original')
axs[2, 1].imshow(cv2.cvtColor(test35, cv2.COLOR_BGR2RGB))
axs[2, 1].set_title('Noisy Image')
axs[2, 2].imshow(cv2.cvtColor(restored35, cv2.COLOR_BGR2RGB))
axs[2, 2].set_title("Wiener, PSNR:"+ str(PSNR(image3,restored35)))
axs[2, 3].imshow(cv2.cvtColor(nlmeans35, cv2.COLOR_BGR2RGB))
axs[2, 3].set_title("Non-Local Means, PSNR:"+ str(PSNR(image3,nlmeans35)))
axs[2, 4].imshow(cv2.cvtColor(bilateral35, cv2.COLOR_BGR2RGB))
axs[2, 4].set_title("Bilateral, PSNR:"+ str(PSNR(image3,bilateral35)))
axs[2, 5].imshow(cv2.cvtColor(median35, cv2.COLOR_BGR2RGB))
axs[2, 5].set_title("Median, PSNR:"+ str(PSNR(image3,median35)))
axs[2, 6].imshow(cv2.cvtColor(gaussian35, cv2.COLOR_BGR2RGB))
axs[2, 6].set_title("Gaussian , PSNR:"+ str(PSNR(image3,gaussian35)))

axs[3, 0].imshow(cv2.cvtColor(image4, cv2.COLOR_BGR2RGB))
axs[3, 0].set_title('Original')
axs[3, 1].imshow(cv2.cvtColor(test45, cv2.COLOR_BGR2RGB))
axs[3, 1].set_title('Noisy Image')
axs[3, 2].imshow(cv2.cvtColor(restored45, cv2.COLOR_BGR2RGB))
axs[3, 2].set_title("Wiener, PSNR:"+ str(PSNR(image4,restored45)))
axs[3, 3].imshow(cv2.cvtColor(nlmeans45, cv2.COLOR_BGR2RGB))
axs[3, 3].set_title("Non-Local Means, PSNR:"+ str(PSNR(image4,nlmeans45)))
axs[3, 4].imshow(cv2.cvtColor(bilateral45, cv2.COLOR_BGR2RGB))
axs[3, 4].set_title("Bilateral, PSNR:"+ str(PSNR(image4,bilateral45)))
axs[3, 5].imshow(cv2.cvtColor(median45, cv2.COLOR_BGR2RGB))
axs[3, 5].set_title("Median, PSNR:"+ str(PSNR(image4,median45)))
axs[3, 6].imshow(cv2.cvtColor(gaussian45, cv2.COLOR_BGR2RGB))
axs[3, 6].set_title("Gaussian , PSNR:"+ str(PSNR(image4,gaussian45)))

for ax in axs.ravel():
    ax.axis('off')

plt.subplots_adjust(wspace=0.1, hspace=0.1)
plt.show()


for ax in axs.ravel():
    ax.axis('off')

plt.subplots_adjust(wspace=0.1, hspace=0.1)
plt.show()

(If the output is appearing as tiny images, double click/click once on the output section. The images are now enlarged and can be viewed by scrolling sideways and downwards.)

# Conclusion:
- Successfully implemented the Wiener filter from scratch for image restoration.
-
Achieved PSNR values ranging from 12 to 17 for most cases, indicating the effectiveness of the restoration
- Comparative analysis showed that the implemented Wiener filter performs at par, if not better as compared to Non-local Means, Bilateral, and Gaussian Smoothing filters in almost all scenarios.
- Exceptional performance of the Median Filter was noted, surpassing all other filters in generating higher quality outputs.
- The conclusions drawn are substantiated by the clear PSNR values obtained for each filter, providing a quantitative measure of image quality.ity.